In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

SEED = 6174
np.random.seed(SEED)

# Set the folder path for data
folder_path = "./input"

In [2]:
de_train = pd.read_parquet(f'{folder_path}/de_train.parquet')
genes = de_train.columns[5:]
id_map = pd.read_csv (f'{folder_path}/id_map.csv')
id_map = id_map.reindex(id_map.columns.tolist() + genes.tolist(), axis=1)

#sm_lincs_id = de_train.set_index('sm_name')["sm_lincs_id"].to_dict()
#sm_name_to_smiles = de_train.set_index('sm_name')['SMILES'].to_dict()

#id_map['sm_lincs_id'] = id_map['sm_name'].map(sm_lincs_id)
#id_map['SMILES'] = id_map['sm_name'].map(sm_name_to_smiles)

de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [3]:
# Drop cell_type = 'T cells CD4+', 'T cells CD8+'
de_train.drop(de_train[de_train['cell_type'] == 'T cells CD4+'].index, inplace = True)
de_train.drop(de_train[de_train['cell_type'] == 'T cells CD8+'].index, inplace = True)

de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
7,T regulatory cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,0.561473,0.149415,3.224920,3.517419,1.361175,...,-0.156127,0.766118,0.702836,0.949482,0.757482,1.163063,1.532419,-0.399292,-2.412165,0.478977
8,B cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.394173,-0.153824,0.178232,0.566241,0.391377,...,-1.052302,-1.176587,-1.220291,-0.278944,-0.095066,1.101790,0.061803,1.406335,-0.264996,-0.119743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,T regulatory cells,Azacitidine,LSM-5218,Nc1ncn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1,False,-0.340365,-0.240775,-0.230234,-0.012160,0.092409,...,0.403436,1.027745,-0.055375,-0.448088,0.161652,-0.016573,0.744808,0.052601,0.691289,0.644323
606,NK cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,0.161760,-0.111839,0.435272,0.021211,-0.120049,...,0.614651,-0.055866,0.265972,-2.065902,-0.049466,-0.169528,-0.011114,0.697897,-0.120168,-0.827473
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760


In [4]:
grouped_de = de_train.groupby('cell_type')[genes].mean().reset_index()
grouped_de

,cell_type,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,B cells,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
1,Myeloid cells,1.570336,0.752564,-2.856826,0.887845,6.658911,4.034911,0.442943,0.403543,0.196285,...,-0.270423,-0.103318,-1.307952,-0.166312,1.883588,0.612681,-0.583563,-0.427938,-0.292768,-0.067723
2,NK cells,0.417735,0.409016,-0.224808,-0.425929,0.282997,1.324508,0.050034,0.405179,-0.065836,...,0.273242,-0.114225,0.019743,0.228173,0.226144,0.366789,0.205059,0.271480,-0.262843,-0.095723
3,T regulatory cells,0.733468,0.442569,0.898680,1.081128,1.145535,1.118455,0.058883,0.585039,-0.005645,...,-0.006385,0.480750,0.251817,0.495385,0.875552,0.873839,0.501520,0.315104,-0.125323,-0.030542


In [5]:
all_sm_names = de_train[de_train["cell_type"]=="B cells"]["sm_name"].to_list()
all_de_train = de_train[de_train["sm_name"].isin(all_sm_names)]
#de_train.drop(all_de_train.index, inplace=True)

all_de_train

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
8,B cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.394173,-0.153824,0.178232,0.566241,0.391377,...,-1.052302,-1.176587,-1.220291,-0.278944,-0.095066,1.101790,0.061803,1.406335,-0.264996,-0.119743
9,Myeloid cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.025146,0.316388,1.366885,1.301593,2.317619,...,-0.902546,-1.445523,0.794772,0.428973,0.605834,0.271988,0.492231,0.354721,1.471559,-0.259483
10,NK cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.861487,-0.112313,-0.355217,0.719999,0.655865,...,0.035687,0.138060,-0.776619,-0.109832,-0.189906,0.753086,0.190892,-0.141699,-0.756510,-0.076934
13,T regulatory cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.210456,0.842411,0.428241,0.556238,0.792801,...,-1.184522,0.020869,0.337240,0.045808,0.318908,0.165729,1.345535,0.370730,-0.640314,-0.135686
30,B cells,Crizotinib,LSM-1027,C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)...,False,-0.221234,-0.588647,0.021279,1.104213,1.584397,...,-0.363383,0.215196,1.177265,0.832644,0.296236,0.603842,1.152725,0.599055,0.253165,0.762857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,T regulatory cells,Oprozomib (ONX 0912),LSM-45496,COC[C@H](NC(=O)c1cnc(C)s1)C(=O)N[C@@H](COC)C(=...,False,0.137616,0.043514,0.041244,0.754874,0.146967,...,0.544100,0.947679,0.640620,0.300745,0.917686,1.669658,2.517226,0.611201,-0.162901,0.100115
556,B cells,CHIR-99021,LSM-1181,Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)c...,False,-3.258689,-0.175129,0.012211,-1.396070,0.103524,...,0.406566,0.023301,0.827045,0.165509,0.400398,-0.371927,-0.476411,0.108601,0.180639,-0.061943
557,Myeloid cells,CHIR-99021,LSM-1181,Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)c...,False,0.338227,-0.431007,0.066335,-0.907600,0.973881,...,0.187125,0.800574,0.214946,-0.104931,-0.125619,-0.160210,-0.886414,-2.955785,-0.866944,-0.053017
558,NK cells,CHIR-99021,LSM-1181,Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)c...,False,-1.026443,-0.024840,0.204772,-0.861985,-2.972540,...,0.323512,1.131738,-0.064157,0.011429,0.030404,0.015837,-0.055027,-0.329874,0.327199,-0.830306


In [6]:
grouped_de = de_train.groupby('cell_type')[genes].mean().reset_index()
grouped_de

,cell_type,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,0.245214,0.279600,-0.203740,-0.362197,0.223396,0.881274,-0.004090,0.323742,-0.080643,...,0.241883,-0.190238,-0.028793,0.154917,0.131314,0.252762,0.217998,0.228114,-0.220319,-0.087720
1,T regulatory cells,0.600773,0.334396,0.713126,0.883802,0.904017,0.880948,0.030486,0.500574,-0.007972,...,-0.004738,0.371276,0.177298,0.356499,0.703001,0.681529,0.391685,0.252294,-0.129672,-0.030756


In [7]:
grouped_all_de = all_de_train.groupby('cell_type')[genes].mean().reset_index()
grouped_all_de

,cell_type,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,B cells,1.380890,0.530585,1.340812,1.594307,4.927551,3.613191,0.028640,0.544636,0.723079,...,0.257778,0.674977,0.217386,1.439374,0.952903,0.581303,0.637408,0.517737,-0.207092,0.079199
1,Myeloid cells,1.570336,0.752564,-2.856826,0.887845,6.658911,4.034911,0.442943,0.403543,0.196285,...,-0.270423,-0.103318,-1.307952,-0.166312,1.883588,0.612681,-0.583563,-0.427938,-0.292768,-0.067723
2,NK cells,1.726861,1.391056,-0.384683,-0.909542,0.735264,4.687871,0.460742,1.023145,0.046517,...,0.511199,0.462575,0.388045,0.784062,0.945736,1.232050,0.106880,0.600556,-0.585519,-0.156448
3,T regulatory cells,1.740391,1.263407,2.306707,2.578485,2.978225,2.920715,0.274367,1.225981,0.012020,...,-0.018877,1.311463,0.817281,1.549286,2.184906,2.333129,1.334974,0.791717,-0.092324,-0.028916


In [8]:
nk_mean = de_train[de_train["cell_type"]=="NK cells"][genes].mean(axis=0)
t_mean = de_train[de_train["cell_type"]=="T regulatory cells"][genes].mean(axis=0)

display(nk_mean)
display(t_mean)

A1BG        0.245214
A1BG-AS1    0.279600
A2M        -0.203740
A2M-AS1    -0.362197
A2MP1       0.223396
              ...   
ZXDB        0.252762
ZXDC        0.217998
ZYG11B      0.228114
ZYX        -0.220319
ZZEF1      -0.087720
Length: 18211, dtype: float64

A1BG        0.600773
A1BG-AS1    0.334396
A2M         0.713126
A2M-AS1     0.883802
A2MP1       0.904017
              ...   
ZXDB        0.681529
ZXDC        0.391685
ZYG11B      0.252294
ZYX        -0.129672
ZZEF1      -0.030756
Length: 18211, dtype: float64

In [9]:
b_mean = ( 
	nk_mean.values + grouped_all_de[grouped_all_de["cell_type"]=="NK cells"][genes].values - grouped_all_de[grouped_all_de["cell_type"]=="B cells"][genes].values +
	t_mean.values + grouped_all_de[grouped_all_de["cell_type"]=="T regulatory cells"][genes].values - grouped_all_de[grouped_all_de["cell_type"]=="B cells"][genes].values
) / 2
b_mean

array([[ 0.77572948,  1.10364489, -0.12510646, ...,  0.41860272,
        -0.30682539, -0.23111925]])

In [10]:
m_mean = (
	nk_mean.values + grouped_all_de[grouped_all_de["cell_type"]=="NK cells"][genes].values - grouped_all_de[grouped_all_de["cell_type"]=="Myeloid cells"][genes].values +
	t_mean.values + grouped_all_de[grouped_all_de["cell_type"]=="T regulatory cells"][genes].values - grouped_all_de[grouped_all_de["cell_type"]=="Myeloid cells"][genes].values
) / 2
m_mean

array([[ 0.58628371,  0.88166522,  4.07253101, ...,  1.36427862,
        -0.22114952, -0.08419706]])

In [15]:
def calc_gene(cell_type, sm_name):
	# Target cell_type and sm_name
	values = []
	rows = de_train[de_train['sm_name'] == sm_name]
	if len(rows) == 0:
		if cell_type == "B cells":
			print("No data: Insert B cells mean")
			return b_mean
		else:
			print("No data: Insert Myeloid cells mean")
			return m_mean

	for index, row in rows.iterrows():
		diff = grouped_all_de[grouped_all_de['cell_type'] == row['cell_type']][genes].values - grouped_all_de[grouped_all_de['cell_type'] == cell_type][genes].values
		values.append(diff + row[genes].values)
	return np.array(values).mean(axis=0)

In [16]:
result = calc_gene("B cells", "Mometasone Furoate")
result

array([[2.778798815733408, 0.8394225438181864, 1.2239385906189892, ...,
        0.03493497934113268, -1.3990084177155255, 0.40592203896409496]],
      dtype=object)

In [17]:
# Add new columsn to id_map genes
for index, row in id_map.iterrows():
	gene_values = calc_gene(row['cell_type'], row['sm_name'])[0]
	for i_g, gene in enumerate(genes):
		id_map.at[index, gene] = gene_values[i_g]

id_map

,id,cell_type,sm_name,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,0.860046,0.463966,0.457143,2.049644,-1.829293,1.534747,0.464007,...,-0.191642,-0.792732,1.255553,-0.471511,1.416221,2.885092,0.765470,0.894158,-0.511477,-0.045026
1,1,B cells,ABT-199 (GDC-0199),0.127368,0.452260,-0.672364,-1.026793,-3.222513,0.078627,-0.148828,...,0.638627,0.522335,0.263866,-0.816487,0.900568,0.750446,0.555067,-0.374447,-0.387551,-0.237000
2,2,B cells,ABT737,0.911963,1.307926,-1.061610,-0.760099,-2.729283,1.606252,0.874979,...,0.406468,-0.341285,0.426114,0.018019,0.536313,2.026019,0.435077,0.304377,0.040202,-0.505083
3,3,B cells,AMD-070 (hydrochloride),0.122956,1.880638,-0.373617,0.245701,-2.570771,0.064535,0.220654,...,-0.368743,-0.355346,0.623696,0.473470,0.569279,1.078916,0.033397,0.424765,-0.363322,-0.025782
4,4,B cells,AT 7867,0.764527,1.012752,-0.420533,-0.227209,-2.595010,0.716110,0.214188,...,-0.571082,0.142462,0.455791,-0.181232,0.977472,2.448288,0.118717,1.014421,-0.366127,0.280203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,Myeloid cells,Vandetanib,0.062346,0.278032,3.229111,-0.767357,-4.994574,-0.621362,-0.106677,...,0.616987,0.793808,1.454852,1.437126,-1.295711,1.358991,1.360760,1.326387,-0.020456,0.650288
251,251,Myeloid cells,Vanoxerine,0.133755,-0.047372,4.306942,0.720015,-4.304868,0.100252,-0.308530,...,0.645568,0.556081,2.155815,1.289953,-0.064514,1.265569,1.326027,1.727886,-0.259213,0.027155
252,252,Myeloid cells,Vardenafil,-0.074975,1.368207,3.578478,-0.335020,-4.940407,-0.788855,-0.207762,...,0.697968,0.679005,1.913313,1.407316,-0.710370,1.297436,0.992962,0.986090,0.306556,0.164257
253,253,Myeloid cells,Vorinostat,1.203560,0.852823,4.756662,0.056412,-3.038654,2.292201,0.213880,...,0.513124,1.487836,1.350306,2.063009,0.777198,2.239984,1.754791,1.047504,-0.284428,0.262200


In [18]:
id_map = id_map.loc[:, ["id"] + genes.to_list()]
id_map.to_csv('submission.csv', index=False)
id_map

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.860046,0.463966,0.457143,2.049644,-1.829293,1.534747,0.464007,0.964525,-0.730593,...,-0.191642,-0.792732,1.255553,-0.471511,1.416221,2.885092,0.765470,0.894158,-0.511477,-0.045026
1,1,0.127368,0.452260,-0.672364,-1.026793,-3.222513,0.078627,-0.148828,0.586213,-0.546562,...,0.638627,0.522335,0.263866,-0.816487,0.900568,0.750446,0.555067,-0.374447,-0.387551,-0.237000
2,2,0.911963,1.307926,-1.061610,-0.760099,-2.729283,1.606252,0.874979,0.907886,-0.959835,...,0.406468,-0.341285,0.426114,0.018019,0.536313,2.026019,0.435077,0.304377,0.040202,-0.505083
3,3,0.122956,1.880638,-0.373617,0.245701,-2.570771,0.064535,0.220654,2.321046,-0.343245,...,-0.368743,-0.355346,0.623696,0.473470,0.569279,1.078916,0.033397,0.424765,-0.363322,-0.025782
4,4,0.764527,1.012752,-0.420533,-0.227209,-2.595010,0.716110,0.214188,1.004187,-0.733263,...,-0.571082,0.142462,0.455791,-0.181232,0.977472,2.448288,0.118717,1.014421,-0.366127,0.280203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.062346,0.278032,3.229111,-0.767357,-4.994574,-0.621362,-0.106677,0.931968,-0.152165,...,0.616987,0.793808,1.454852,1.437126,-1.295711,1.358991,1.360760,1.326387,-0.020456,0.650288
251,251,0.133755,-0.047372,4.306942,0.720015,-4.304868,0.100252,-0.308530,0.695667,-0.404707,...,0.645568,0.556081,2.155815,1.289953,-0.064514,1.265569,1.326027,1.727886,-0.259213,0.027155
252,252,-0.074975,1.368207,3.578478,-0.335020,-4.940407,-0.788855,-0.207762,0.476237,-0.001246,...,0.697968,0.679005,1.913313,1.407316,-0.710370,1.297436,0.992962,0.986090,0.306556,0.164257
253,253,1.203560,0.852823,4.756662,0.056412,-3.038654,2.292201,0.213880,2.102906,1.436193,...,0.513124,1.487836,1.350306,2.063009,0.777198,2.239984,1.754791,1.047504,-0.284428,0.262200
